In [1]:
using BenchmarkTools
using FLoops

using Distributed



https://discourse.julialang.org/t/questions-about-getting-started-with-parallel-computing/25341

Distributed

In [3]:
nprocs()

1

In [4]:
addprocs(4)

4-element Array{Int64,1}:
 2
 3
 4
 5

In [5]:
nprocs()

5

In [15]:
@everywhere using NBInclude
@everywhere @nbinclude("solve_model.ipynb")

In [12]:
#Vh/D, L/D, Dj/D
@everywhere data = [1.0, 0.25, 0.68]; 

#ρ, β, c0, λj
@everywhere game_parameters = [0.884, 9.84, 0.044, 0.346];

In [9]:
@everywhere function solve_tree_serial(n, data=data, game_parameters=game_parameters)
    for i in 1:n
        solve_tree(data, game_parameters)
        
    end
end

    
    

In [10]:
function solve_tree_parallel(n)
    pmap(solve_tree_serial, n)
end


solve_tree_parallel (generic function with 1 method)

In [16]:
@time solve_tree_parallel(2)

      From worker 4:	303.242164 seconds (635.94 M allocations: 210.131 GiB, 8.79% gc time)
      From worker 4:	330.995468 seconds (632.36 M allocations: 209.969 GiB, 10.06% gc time)
636.397515 seconds (45.48 k allocations: 1.868 MiB)


In [20]:
function solve_tree_parallel(n)
    pmap(solve_tree_serial, n, distributed=true)
end

solve_tree_parallel (generic function with 1 method)

In [21]:
@time solve_tree_parallel(2)

      From worker 3:	310.403359 seconds (635.94 M allocations: 210.131 GiB, 8.73% gc time)
      From worker 3:	325.235513 seconds (632.36 M allocations: 209.969 GiB, 10.36% gc time)
638.206392 seconds (19.76 k allocations: 585.562 KiB)


In [22]:
procs()

5-element Array{Int64,1}:
 1
 2
 3
 4
 5

In [23]:
@time solve_tree_parallel([1,1])

      From worker 2:	346.466306 seconds (635.94 M allocations: 210.131 GiB, 8.67% gc time)
      From worker 5:	360.064234 seconds (635.94 M allocations: 210.131 GiB, 9.48% gc time)
362.812039 seconds (195.62 k allocations: 10.088 MiB)


2-element Array{Nothing,1}:
 nothing
 nothing

#### Teste: gerar draws da uniforme

In [ ]:
@btime rand(1000);

In [ ]:
@btime for i in 1:1000
    rand()
end

In [ ]:
@btime @floop for i in 1:1000
    rand()
end

In [ ]:
@btime @distributed for i in 1:1000
    rand()
end

Somando os draws

In [ ]:
function draws(ndraws, ncores)
    
    v=0
    
    for n in 1:ndraws
        v+=rand()
    end
    
end   

In [ ]:
@btime draws(1000, 1)

In [ ]:
function draws_floop(ndraws, ncores)
    
    v=0
    
    #parece importante usar ÷, se usar \ dá um erro
    @floop ThreadedEx(basesize = ndraws ÷ ncores) for n in 1:ndraws
        @reduce(v+=rand())
    end
end

In [ ]:
@btime draws_floop(1000, 1)

In [ ]:
@btime draws_floop(1000, 2)

In [ ]:
@btime draws_floop(1000, 3)

In [ ]:
@btime draws_floop(1000, 4)

Conclusão: ou estou fazendo errado ou operações com rand() ficam melhores usando apenas 1 core

#### Teste: somar valores em um loop

In [ ]:
function somar(n)
    s=0
    for i in 1:n
        s+=i
    end
    return s
end
        

In [ ]:
@btime somar(1_000_000_000)

In [ ]:
function somar_floop(n, executor)
    s=0
    @floop executor for i in 1:n
        @reduce(s+=i)
    end
    return s
end


base = 2;
ex1 = SequentialEx(simd = Val(true));
ex2 = ThreadedEx(basesize = base)
ex3 = DistributedEx(threads_basesize = base);

In [ ]:
@btime somar_floop(10, ex1)

In [ ]:
@btime somar_floop(10, ex2)

In [ ]:
@btime somar_floop(10, ex3)

## Threads

In [ ]:
#mudei a variavel ambiente JULIA_NUM_THREADS no windows 10
Threads.nthreads()

### Exemplo de rodar o código da solução do modelo

In [ ]:
#Vh/D, L/D, Dj/D
data = [1.0, 0.25, 0.68]; 

#ρ, β, c0, λj
game_parameters = [0.884, 9.84, 0.044, 0.346];

Sem paralelizar

@time for i in 1:2
    solve_tree(data, game_parameters);
end
    

Com paralelização

@time @floop ThreadedEx(basesize=1) for i in 1:2
    solve_tree(data, game_parameters);
end

@time @floop ThreadedEx(basesize=1) for i in 1:4
    solve_tree(data, game_parameters);
end

### Distributed

In [ ]:
nprocs()

In [ ]:
addprocs(4)

In [ ]:
nprocs()

In [ ]:
@sync @distributed for i in 1:2
    solve_tree(data, game_parameters)
end